# Mask R-CNN - Train on Shapes Dataset


This notebook shows how to train Mask R-CNN on your own dataset. To keep things simple we use a synthetic dataset of shapes (squares, triangles, and circles) which enables fast training. You'd still need a GPU, though, because the network backbone is a Resnet101, which would be too slow to train on a CPU. On a GPU, you can start to get okay-ish results in a few minutes, and good results in less than an hour.

The code of the *Shapes* dataset is included below. It generates images on the fly, so it doesn't require downloading any data. And it can generate images of any size, so we pick a small image size to train faster. 

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

# Root directory of the project
ROOT_DIR = os.path.abspath("./")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

Using TensorFlow backend.


## Configurations

In [2]:
from brain import BrainsConfig
config = BrainsConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     8
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 8
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  256
IMAGE_META_SIZE                24
IMAGE_MIN_DIM                  256
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [256 256   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE             

## Notebook Preferences

In [3]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Dataset

Create a synthetic dataset

Extend the Dataset class and add a method to load the shapes dataset, `load_shapes()`, and override the following methods:

* load_image()
* load_mask()
* image_reference()

In [4]:
from preprocess import dataset_preprocess
dataset_preprocess()

Total data number is 6727
Training data number is 5381
Val data number is 1346


In [5]:
from brain import BrainsDataset

# Dataset path
train_data_path = './../dataset/Processed/train'
val_data_path = './../dataset/Processed/val'

# Training dataset
dataset_train = BrainsDataset()
dataset_train.load_brains(train_data_path)
dataset_train.prepare()


# Validation dataset
dataset_val = BrainsDataset()
dataset_val.load_brains(val_data_path)
dataset_val.prepare()

## Create Model

In [6]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config, model_dir=MODEL_DIR)

In [7]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

## Training

Train in two stages:
1. Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass `layers='heads'` to the `train()` function.

2. Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass `layers="all` to train all layers.

In [8]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=1, 
            layers='heads')

Instructions for updating:
Use the retry module or similar alternatives.

Starting at epoch 0. LR=0.001

Checkpoint Path: /Users/dodo/Documents/projects/maskrcnn/train/logs/brain20181204T0108/mask_rcnn_brain_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistribute

/Users/dodo/.conda/envs/py3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/Users/dodo/.conda/envs/py3/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/1


ERROR:root:Error processing image {'id': 5177, 'source': 'brains', 'path': './../dataset/Processed/train/5_t1_w2r_40.npy'}
Traceback (most recent call last):
  File "/Users/dodo/Documents/projects/maskrcnn/train/mrcnn/model.py", line 1710, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/Users/dodo/Documents/projects/maskrcnn/train/mrcnn/model.py", line 1213, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "/Users/dodo/Documents/projects/maskrcnn/train/brain.py", line 106, in load_mask
    each_class = np.ones(seg.shape[0], seg.shape[1])
  File "/Users/dodo/.conda/envs/py3/lib/python3.6/site-packages/numpy/core/numeric.py", line 188, in ones
    a = empty(shape, dtype, order)
TypeError: data type not understood
ERROR:root:Error processing image {'id': 1797, 'source': 'brains', 'path': './../dataset/Processed/train/148_t4_w1r_26.npy'}
Traceback (most recent call last):
  File "/Users/dodo/Documents/projects/maskrcnn/train/mrcnn/model.py", line

TypeError: data type not understood
ERROR:root:Error processing image {'id': 780, 'source': 'brains', 'path': './../dataset/Processed/val/7_t2_w1_18.npy'}
Traceback (most recent call last):
  File "/Users/dodo/Documents/projects/maskrcnn/train/mrcnn/model.py", line 1710, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/Users/dodo/Documents/projects/maskrcnn/train/mrcnn/model.py", line 1213, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "/Users/dodo/Documents/projects/maskrcnn/train/brain.py", line 106, in load_mask
    each_class = np.ones(seg.shape[0], seg.shape[1])
  File "/Users/dodo/.conda/envs/py3/lib/python3.6/site-packages/numpy/core/numeric.py", line 188, in ones
    a = empty(shape, dtype, order)
TypeError: data type not understoodERROR:root:Error processing image {'id': 1797, 'source': 'brains', 'path': './../dataset/Processed/train/148_t4_w1r_26.npy'}
Traceback (most recent call last):
  File "/Users/dodo/Documents/projects/maskr

...

TypeError: data type not understoodERROR:root:Error processing image {'id': 1414, 'source': 'brains', 'path': './../dataset/Processed/train/148_t2_w1r_27.npy'}
Traceback (most recent call last):
  File "/Users/dodo/Documents/projects/maskrcnn/train/mrcnn/model.py", line 1710, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/Users/dodo/Documents/projects/maskrcnn/train/mrcnn/model.py", line 1213, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "/Users/dodo/Documents/projects/maskrcnn/train/brain.py", line 106, in load_mask
    each_class = np.ones(seg.shape[0], seg.shape[1])
  File "/Users/dodo/.conda/envs/py3/lib/python3.6/site-packages/numpy/core/numeric.py", line 188, in ones
    a = empty(shape, dtype, order)
TypeError: data type not understood
ERROR:root:Error processing image {'id': 5362, 'source': 'brains', 'path': './../dataset/Processed/train/5_t2_w2r_34.npy'}
Traceback (most recent call last):
  File "/Users/dodo/Documents/projects/m

TypeError: data type not understoodERROR:root:Error processing image {'id': 4288, 'source': 'brains', 'path': './../dataset/Processed/train/4_t2_w1r_19.npy'}
Traceback (most recent call last):
  File "/Users/dodo/Documents/projects/maskrcnn/train/mrcnn/model.py", line 1710, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/Users/dodo/Documents/projects/maskrcnn/train/mrcnn/model.py", line 1213, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "/Users/dodo/Documents/projects/maskrcnn/train/brain.py", line 106, in load_mask
    each_class = np.ones(seg.shape[0], seg.shape[1])
  File "/Users/dodo/.conda/envs/py3/lib/python3.6/site-packages/numpy/core/numeric.py", line 188, in ones
    a = empty(shape, dtype, order)
TypeError: data type not understood
ERROR:root:Error processing image {'id': 1197, 'source': 'brains', 'path': './../dataset/Processed/val/7_t4_w1_5.npy'}
Traceback (most recent call last):
  File "/Users/dodo/Documents/projects/maskrcn

TypeError: data type not understood




ERROR:root:Error processing image {'id': 4288, 'source': 'brains', 'path': './../dataset/Processed/train/4_t2_w1r_19.npy'}
Traceback (most recent call last):
  File "/Users/dodo/Documents/projects/maskrcnn/train/mrcnn/model.py", line 1710, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/Users/dodo/Documents/projects/maskrcnn/train/mrcnn/model.py", line 1213, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "/Users/dodo/Documents/projects/maskrcnn/train/brain.py", line 106, in load_mask
    each_class = np.ones(seg.shape[0], seg.shape[1])
  File "/Users/dodo/.conda/envs/py3/lib/python3.6/site-packages/numpy/core/numeric.py", line 188, in ones
    a = empty(shape, dtype, order)
TypeError: data type not understood
ERROR:root:Error processing image {'id': 570, 'source': 'brains', 'path': './../dataset/Processed/val/7_orig_w2r_45.npy'}
Traceback (most recent call last):
  File "/Users/dodo/Documents/projects

TypeError: data type not understoodERROR:root:Error processing image {'id': 4288, 'source': 'brains', 'path': './../dataset/Processed/train/4_t2_w1r_19.npy'}
Traceback (most recent call last):
  File "/Users/dodo/Documents/projects/maskrcnn/train/mrcnn/model.py", line 1710, in data_generator
    use_mini_mask=config.USE_MINI_MASK)
  File "/Users/dodo/Documents/projects/maskrcnn/train/mrcnn/model.py", line 1213, in load_image_gt
    mask, class_ids = dataset.load_mask(image_id)
  File "/Users/dodo/Documents/projects/maskrcnn/train/brain.py", line 106, in load_mask
    each_class = np.ones(seg.shape[0], seg.shape[1])
  File "/Users/dodo/.conda/envs/py3/lib/python3.6/site-packages/numpy/core/numeric.py", line 188, in ones
    a = empty(shape, dtype, order)
TypeError: data type not understoodERROR:root:Error processing image {'id': 470, 'source': 'brains', 'path': './../dataset/Processed/val/7_orig_w1r_41.npy'}
Traceback (most recent call last):
  File "/Users/dodo/Documents/projects/maskr

TypeError: data type not understood

In [ ]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=2, 
            layers="all")

In [ ]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
# model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
# model.keras_model.save_weights(model_path)

## Detection

In [ ]:
class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig()

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [ ]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_class_id, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_class_id", gt_class_id)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox, gt_mask, gt_class_id, 
                            dataset_train.class_names, figsize=(8, 8))

In [ ]:
results = model.detect([original_image], verbose=1)

r = results[0]
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

## Evaluation

In [ ]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
image_ids = np.random.choice(dataset_val.image_ids, 10)
APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox, gt_class_id, gt_mask,
                         r["rois"], r["class_ids"], r["scores"], r['masks'])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))